In [ ]:
import numpy as np
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, BatchNormalization, MaxPooling2D, Flatten, Dense, Dropout, LSTM
from tensorflow.keras.models import Model
import os

# Define input shape
input_shape = (224, 224, 3)

# Define the input layer
input_layer = Input(shape=input_shape)

# Define the convolutional base using ResNet50
conv_base = tf.keras.applications.ResNet50(include_top=False, weights='imagenet', input_shape=input_shape)(input_layer)

# Add BatchNormalization and MaxPooling layers
x = BatchNormalization()(conv_base)
x = MaxPooling2D(pool_size=(2, 2))(x)

# Flatten the output of the convolutional base
x = Flatten()(x)

# Reshape the output to have a time dimension
x = tf.keras.layers.Reshape((1, -1))(x)

# Define the LSTM layer
x = LSTM(128)(x)

# Add a Dense layer with 64 units and ReLU activation
x = Dense(64, activation='relu')(x)

# Add a Dropout layer with a rate of 0.5
x = Dropout(0.5)(x)

# Add the output layer with a sigmoid activation
output_layer = Dense(1, activation='sigmoid')(x)

# Define the model
model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Define file paths to your training and validation data directories
train_directory = '/Users/nithishkumar/Documents/deepfake-video-detection-project/final_dataset/train'
val_directory = '/Users/nithishkumar/Documents/deepfake-video-detection-project/final_dataset/validation'

# Define train_labels
train_labels = [0] * len(os.listdir(os.path.join(train_directory, 'original')) + [1] * len(os.listdir(os.path.join(train_directory, 'deepfake'))))
val_labels = [0] * len(os.listdir(os.path.join(val_directory, 'original')) + [1] * len(os.listdir(os.path.join(val_directory, 'deepfake'))))

# Use data generators to load and preprocess the images
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define data generators for training and validation data
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1.0/255.0)

# Load and preprocess training and validation data using data generators
batch_size = 32

train_generator = train_datagen.flow_from_directory(
    train_directory,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='binary'
)

validation_generator = val_datagen.flow_from_directory(
    val_directory,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='binary'
)

# Train the model
model.fit(train_generator, validation_data=validation_generator, epochs=10)

# Your model is now trained and ready for making predictions.


Found 970 images belonging to 2 classes.
Found 206 images belonging to 2 classes.
Epoch 1/10
31/31 [==============================] - 150s 5s/step - loss: 0.2056 - accuracy: 0.9258 - val_loss: 1.0453 - val_accuracy: 0.5000
Epoch 2/10
31/31 [==============================] - 160s 5s/step - loss: 0.1607 - accuracy: 0.9443 - val_loss: 0.8582 - val_accuracy: 0.5000
Epoch 3/10
31/31 [==============================] - 173s 6s/step - loss: 0.1285 - accuracy: 0.9588 - val_loss: 1.3425 - val_accuracy: 0.5000
Epoch 4/10
31/31 [==============================] - 164s 5s/step - loss: 0.1254 - accuracy: 0.9598 - val_loss: 1.0047 - val_accuracy: 0.5000
Epoch 5/10
31/31 [==============================] - 158s 5s/step - loss: 0.1305 - accuracy: 0.9577 - val_loss: 0.9753 - val_accuracy: 0.5000
Epoch 6/10
31/31 [==============================] - 163s 5s/step - loss: 0.1179 - accuracy: 0.9598 - val_loss: 0.9815 - val_accuracy: 0.5000
Epoch 7/10
31/31 [==============================] - 170s 5s/step - loss:

In [ ]:
# Save the model to a file
model.save("deepfake_detection_model.h5")


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
